In [1]:
import glob
import json
import torch
import numpy as np
from PIL import Image
from data import visualize
from data import data_utils
from data import nerf_helpers
from model import models
from matplotlib import pyplot as plt

In [2]:
# separating out the poses for each image
# transforms_file = '/home/parika/WorkingDir/Thesis/Data/nerf_synthetic/lego/transforms_test.json'
# out_dir = '/home/parika/WorkingDir/Thesis/Data/nerf_synthetic/lego/test/'

# with open(transforms_file, 'r') as fp:
#     meta = json.load(fp)

# for frame in meta['frames']:
#     frame_meta = {}
#     file_path = frame['file_path']
#     fname = file_path[file_path.rfind('/')+1:]
#     frame_meta['pose'] = frame['transform_matrix']
#     frame_meta['rotation'] = frame['rotation']

#     out_file = out_dir + fname + '.json'

#     with open(out_file, 'w') as cam_file:
#         json.dump(frame_meta, cam_file)


            

In [3]:
# converting camera information to the format used for ejecta so we can use the same code
datadir = '/home/parika/WorkingDir/Thesis/Data/nerf_synthetic/lego/test/'
savedir = '/home/parika/WorkingDir/Thesis/Data/nerf_synthetic/lego/cam/'

for cam_file in glob.glob(os.path.join(datadir,"*.json")):
    fname = cam_file[cam_file.rfind('/')+1:cam_file.rfind('.')]
    new_camfile = os.path.join(savedir, fname+".json")
    info = {}

    with open(cam_file, 'r') as fp:
        cam_info = json.load(fp)

    pose = torch.Tensor(cam_info['pose']).float()

    origin_in_cam = torch.Tensor([0.0, 0.0, 0.0, 1.0])
    origin_in_world = pose @ origin_in_cam
    spherical = data_utils.cartesian_to_spherical(origin_in_world.unsqueeze_(0))

    cartesian_on_radius1 = data_utils.spherical_to_cartesian(torch.Tensor([0.0,0.0,0.0]),1.2,spherical[0,0],
    spherical[0,1]) 

    info['pose'] = cam_info['pose']
    info['rotation_angle_x'] = cam_info['rotation']

    # actual origin calculated from pose matrix
    info['actual_origin'] = origin_in_world.tolist()

    # on the same angular line, we calculated point on sphere with radius 1.2
    # we will use this one for training, again to keep it similar across all datasets
    info['origin'] = cartesian_on_radius1.tolist()

    # we will train at half resolution
    info['width'] = 400
    info['height'] = 400

    # rotation_angle_x in degrees is also 40 (maybe it is fov)
    # and we are taking same params as used for ejecta
    info['fov'] = 40.0
    info['farZ'] = 10.0
    info['nearZ'] = 0.10000000149011612
    info['lookAt'] = [0.0,0.0,0.0]
    info['up'] = [0.0,1.0,0.0]

    with open(new_camfile, 'w') as fp:
        json.dump(info, fp)

KeyError: 'rotation'

In [4]:
# projecting on volume

# volume params
center = torch.Tensor([0.0, 0.0, 0.0])
radius = 1.0
vol_params = ("sphere", center, radius)

datadir = '/home/parika/WorkingDir/Thesis/Data/nerf_synthetic/lego/train/'
savedir = '/home/parika/WorkingDir/Thesis/Data/nerf_synthetic/lego/train_vis/'

for cam_file in sorted(glob.glob(os.path.join(datadir,"*.json"))):
    fname = cam_file[cam_file.rfind('/')+1:cam_file.rfind('.')]
    img_file = os.path.join(datadir,fname+'.png')
    vis_file = os.path.join(savedir,fname+'_radius4.ply')

    print(cam_file)
    print(img_file)
    print(vis_file)

    points, viewdirs, valid_mask = data_utils.get_input_data(cam_file, vol_params, "cartesian", "cartesian", None)

    img = Image.open(img_file)
    img = img.resize(size=(400, 400), resample=Image.NEAREST)
    img = torch.Tensor(np.array(img).astype(np.float32)[...,:3] / 255.).permute(2,0,1)

    # points[1,...] = -points[1,...]
    # points[2,...] = -points[2,...]

    visualize.project_input_on_bounding_vol(points, img, valid_mask, vis_file)

/home/parika/WorkingDir/Thesis/Data/nerf_synthetic/lego/train/r_0.json
/home/parika/WorkingDir/Thesis/Data/nerf_synthetic/lego/train/r_0.png
/home/parika/WorkingDir/Thesis/Data/nerf_synthetic/lego/train_vis/r_0_radius4.ply
No face information provided. Saving point cloud. To visualize voxelized mesh, please provide face information.
/home/parika/WorkingDir/Thesis/Data/nerf_synthetic/lego/train/r_1.json
/home/parika/WorkingDir/Thesis/Data/nerf_synthetic/lego/train/r_1.png
/home/parika/WorkingDir/Thesis/Data/nerf_synthetic/lego/train_vis/r_1_radius4.ply
No face information provided. Saving point cloud. To visualize voxelized mesh, please provide face information.
/home/parika/WorkingDir/Thesis/Data/nerf_synthetic/lego/train/r_10.json
/home/parika/WorkingDir/Thesis/Data/nerf_synthetic/lego/train/r_10.png
/home/parika/WorkingDir/Thesis/Data/nerf_synthetic/lego/train_vis/r_10_radius4.ply
No face information provided. Saving point cloud. To visualize voxelized mesh, please provide face infor

In [1]:
import glob
import json
import torch
import numpy as np
from PIL import Image
from data import visualize
from data import lego_helpers as helpers

# volume params
center = torch.Tensor([0.0, 0.0, 0.0])
radius = 3.0
vol_params = ("sphere", center, radius)

hwf = [800, 800, 1111.1110311937682]
near = 2.0
far = 6.0
datadir = '/home/parika/WorkingDir/Thesis/Data/nerf_synthetic/lego/train'
savedir = '/home/parika/WorkingDir/Thesis/Data/nerf_synthetic/lego/vis'

for cam_file in sorted(glob.glob(os.path.join(datadir,"r_0.json"))):
    fname = cam_file[cam_file.rfind('/')+1:cam_file.rfind('.')]

    img_file = os.path.join(datadir,fname+'.png')
    ply_file=os.path.join(savedir,fname+'_2.ply')
    with open(cam_file,'r') as fp:
        meta = json.load(fp)
    pose = torch.Tensor(meta['pose'])

    points, viewdirs, valid_mask = helpers.project_onto_vol(vol_params, hwf, pose, near, far, 'cartesian')

    img = Image.open(img_file)
    img= np.array(img).astype(np.float32)[...,:3] / 255.
    img[~valid_mask] = 0.0
    img = torch.Tensor(img)

    visualize.project_input_on_bounding_vol(points.permute(2,0,1), 
    img.permute(2,0,1), valid_mask, ply_file=ply_file)

No face information provided. Saving point cloud. To visualize voxelized mesh, please provide face information.


In [28]:
# separate tensorboard images into individual images
from PIL import Image
img_file = '/home/goel/Downloads/gt.png'
img = Image.open(img_file)

first_img = img.crop((0,0,400,400))
first_img.save('/home/goel/Downloads/img5.png')

first_img = img.crop((401,0,800,400))
first_img.save('/home/goel/Downloads/img6.png')

first_img = img.crop((801,0,1200,400))
first_img.save('/home/goel/Downloads/img7.png')

first_img = img.crop((1201,0,1600,400))
first_img.save('/home/goel/Downloads/img8.png')

In [10]:
# combining two images into one vertically
images = [Image.open(x) for x in ['/home/goel/Downloads/pred1.png', '/home/goel/Downloads/pred2.png', '/home/goel/Downloads/pred3.png', '/home/goel/Downloads/pred4.png']]
widths, heights = zip(*(i.size for i in images))

total_width = sum(widths)
max_height = max(heights)

new_im = Image.new('RGB', (total_width, max_height))

x_offset = 0
for im in images:
  new_im.paste(im, (x_offset,0))
  x_offset += im.size[0]

new_im.save('/home/goel/Downloads/nomap_train.png')

In [2]:
# global params

width = 512
height = 512
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# volume params
center = torch.Tensor([0.0, 0.0, 0.0])
radius = 1.0
vol_params = ("sphere", center, radius)

np.random.seed(42)
torch.manual_seed(42)

In [4]:
# load saved model
model_name = 'ConvNet'
input_ch = 6
points_type = 'cartesian'
viewdir_type = 'cartesian'
out_dir = '/home/goel/Thesis/Code/dvr/outputs/lego/multiviews/no_map_radius1_mse/'
writer_path = os.path.join(out_dir,'logs')
model_path = os.path.join(out_dir,'models/best_model_194.pth')

# create the model
model = models.ConvNet(input_ch= input_ch,output_ch=3).to(device)

checkpoint = torch.load(model_path, map_location=device)
input_map = checkpoint['input_map']
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [9]:
data_dir = '/home/goel/Thesis/Data/nerf_synthetic/lego/train/'

gt_file = os.path.join(data_dir,"r_88.png")
cam_file = os.path.join(data_dir,"r_88.json")

gt, points, viewdirs, valid_mask = nerf_helpers.get_input_data(gt_file, cam_file, vol_params, True, 0.6911112070083618, 2.0, 6.0, "cartesian", "cartesian") 
input = torch.cat((points, viewdirs),dim=0)
input = data_utils.input_mapping(input, input_map, True, True, "cartesian", "ConvNet")
input = input.unsqueeze_(0).to(device)
output = model(input)
output = torch.clamp(output,min=0.0,max=1.0)

# gt_img = (gt.permute((1,2,0))*255.0).numpy().astype(np.uint8)
# gt_img = Image.fromarray(gt_img)
# gt_img.save(out_dir+'gt13.png')

pred_img = (output.detach().cpu()[0].permute((1,2,0))*255.0).numpy().astype(np.uint8)
pred_img = Image.fromarray(pred_img)
pred_img.save('/home/goel/Downloads/pred4.png')

In [40]:
data_dir = '/home/goel/Thesis/Data/nerf_synthetic/lego/train/'
for gt_file in glob.glob(data_dir+'*.png'):
    idx = gt_file[gt_file.rfind('/')+1:gt_file.rfind('.')]
    cam_file = data_dir+idx+'.json'
    gt, points, viewdirs, valid_mask = nerf_helpers.get_input_data(gt_file, cam_file, vol_params, True, 0.6911112070083618, 2.0, 6.0, "cartesian", "cartesian")
    gt[:,~valid_mask] = 0.0
    gt_img = (gt.permute((1,2,0))*255.0).numpy().astype(np.uint8)
    gt_img = Image.fromarray(gt_img)
    gt_img.save('/home/goel/Downloads/gt_files/'+idx+'.png')

/home/goel/Thesis/Data/nerf_synthetic/lego/train/r_56.png
/home/goel/Thesis/Data/nerf_synthetic/lego/train/r_82.png
/home/goel/Thesis/Data/nerf_synthetic/lego/train/r_28.png
/home/goel/Thesis/Data/nerf_synthetic/lego/train/r_7.png
/home/goel/Thesis/Data/nerf_synthetic/lego/train/r_97.png
/home/goel/Thesis/Data/nerf_synthetic/lego/train/r_23.png
/home/goel/Thesis/Data/nerf_synthetic/lego/train/r_91.png
/home/goel/Thesis/Data/nerf_synthetic/lego/train/r_22.png
/home/goel/Thesis/Data/nerf_synthetic/lego/train/r_67.png
/home/goel/Thesis/Data/nerf_synthetic/lego/train/r_18.png
/home/goel/Thesis/Data/nerf_synthetic/lego/train/r_41.png
/home/goel/Thesis/Data/nerf_synthetic/lego/train/r_32.png
/home/goel/Thesis/Data/nerf_synthetic/lego/train/r_44.png
/home/goel/Thesis/Data/nerf_synthetic/lego/train/r_31.png
/home/goel/Thesis/Data/nerf_synthetic/lego/train/r_10.png
/home/goel/Thesis/Data/nerf_synthetic/lego/train/r_39.png
/home/goel/Thesis/Data/nerf_synthetic/lego/train/r_38.png
/home/goel/Thes

In [38]:
gt.shape

torch.Size([3, 400, 400])